## Container: torchfort_v4.sif

In [1]:
%%bash
singularity shell --nv $HOME/containers/torchfort_v4.sif
nvidia-smi
exit

Thu May 15 13:23:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           On  |   00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0             25W /  250W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Equivalences Docker (D) vs Singularity (S)

    --gpus=all → On Singularity, we use --nv to enable NVIDIA GPUs.
    -v /host:/container → On Singularity, we use --bind /host:/container.
    -p → Singularity doesn't manage ports directly like Docker, so it needs alternatives like external SSH redirection.
    --user 1000:1000 → Singularity inherits the system user, so there is no need to set a user ID.


| **Docker**                        | **Singularity**                          |
|------------------------------------|------------------------------------------|
| `docker run -d --name t01 image`  | `singularity instance start image.sif t01` |
| `docker exec -it t01 bash`        | `singularity exec instance://t01 bash` |
| `docker stop t01`                 | `singularity instance stop t01` |

- In Singularity, when stopping the instance, the changes are lost. You need to save first with the command: `singularity build new_container.sif torchfort_v4.sif`
- Singularity automatically mounts the host directories: `$HOME, $PWD, /tmp, /var/tmp, /proc, /sys, /dev, /etc/resolv.conf, /etc/passwd, /etc/group`
- In Singularity, containers run in the same network namespace as the host system, so access can be more direct than in containers like Docker.
- On Singularity, by default, images (.sif) are read-only. To be able to write to them, you can mount directories on the host using --bind, use a writable tmpfs overlay filesystem that allocates a small amount of RAM for this purpose with --writable-tmpfs, create an overlay (like a union filesystem) with an external file and using --overlay, and create a directory as a container using --sandbox.
- The Singularity container automatically mounts the instance's `$HOME` as the host's `$HOME`, so it ends up mixing files used by both the container and the host in `$HOME`. This behavior may not always be desirable, especially if the operating systems and software used in the container and the host are different. An example is the conda environment, which may exist in the host's `$HOME` and another environment in the container. In this case it would be better to use the `--no-home` flag and mount the host's `$HOME` in `mnt/`, e.g. with the `--bind $HOME:/mnt/x` flag.

Interactive:

singularity run --nv $HOME/containers/torchfort_v4.sif

Background run:

In [3]:
! singularity instance start --nv $HOME/containers/torchfort_v4.sif t01

INFO:    Instance stats will not be available - system configuration does not support cgroup management.
INFO:    instance started successfully


Other commands:

In [5]:
! singularity exec instance://t01 pwd

/prj/ampemi/eduardo.miranda2/radnn/src/torchfort


In [7]:
%%bash
singularity exec instance://t01 \
    jupyter-lab \
    --no-browser \
    --notebook-dir=/ \
    --IdentityProvider.token='' --ServerApp.password='' \
    --ip=0.0.0.0 --port=8895 \
    >/dev/null 2>&1 &

In [2]:
! singularity instance stop t01

INFO:    Stopping t01 instance of /home/x/containers/torchfort_v05.sif (PID=4914)


In [4]:
! singularity instance list

INSTANCE NAME    PID        IP    IMAGE
t01              2239860          /prj/ampemi/eduardo.miranda2/containers/torchfort_v4.sif


In [ ]:
! singularity build new_container.sif torchfort_v4.sif